# Description 

1. get all fruits that the player can load either alone or with others
2. for each of these fruits calculate summery statistics
3. for each player make a prediciton for each of the fruits and save the result in a dictionary for each player. id fruits and prob
4. given the first order believes about the othe players, choose the next target and give teh result as training back to teh prediction for the player
5. When another player loads a fruit, train the neural network with the results by mapping it with the id. 

So the nn is trained to predict the reqard maximizing choice of a player given the choices of other players

## Notes

- reward = level_player * level_food creating a clear incentive to cooperate

# Code

### Setup

#### Requirements

In [5]:
import lbforaging
import gymnasium as gym
from lbforaging.agents import H1
import time
import numpy as np

from tr_lbf_addon.lbf_gym import Lbf_Gym

#### Create the enviromente

In [6]:
# register the environment in gym

field_size = 8 # size of the game board
number_players = 2 # Number of players
max_num_food = 8 # max amount of food on the board. TODO How is teh amount of food determined?
coop_mode = False # If true, all foods will have teh max level so that all foods can only be loaded by working with other players
max_episode_steps = 50 # Number of steps until one round (episode) is terminated
sight = 0  #  How far can the agents see i presume TODO
max_player_level = 1
min_player_level = 1
max_food_level = 1
min_food_level = 1
normalize_reward = True
grid_observation = False
observe_agent_levels = True # If true, the observation will include the level of the agents
penalty = 0.0 # if the player was not the one to load the food, it gets a penalty to its reward
render_mode = "human"
full_info_mode = True

id_string = "Foraging-{0}x{0}-{1}p-{2}f{3}-v3".format(field_size, number_players, max_num_food, "-coop" if coop_mode else "")

gym.envs.registration.register(
    id=id_string,
    entry_point="lbforaging.foraging:ForagingEnv",
    kwargs={
        "players": number_players,
        "max_player_level": max_player_level,
        "min_player_level": min_player_level, 
        "max_food_level": max_food_level,
        "min_food_level": min_food_level,
        "field_size": (field_size, field_size),
        "max_num_food": max_num_food,
        "sight": sight,
        "max_episode_steps": max_episode_steps,
        "force_coop": coop_mode,
        "normalize_reward" : normalize_reward,
        "grid_observation" : grid_observation,
        "observe_agent_levels" : observe_agent_levels,
        "penalty" : penalty,
        "render_mode" : render_mode,
        "full_info_mode": full_info_mode
        },
    )

In [7]:
# define the environment. A more detailed way is discribed on https://github.com/semitable/lb-foraging
env = gym.make(id_string) # "Foraging-{GRID_SIZE}x{GRID_SIZE}-{PLAYER COUNT}p-{FOOD LOCATIONS}f{-coop IF COOPERATIVE MODE}-v0"
# render_mode is "human" per default

# reset the environment with a seed
observation, info = env.reset(seed=42)

# initialize the class
tr_marla_class = Lbf_Gym(observation[0])

In [30]:
import itertools
import pandas as pd

levels = [8,2,6,4]

duo_coop_levels = pd.Series([np.sum(levels) for levels in list(itertools.combinations(levels, 2))]).unique().tolist()
tripple_coop_levels = pd.Series([np.sum(levels) for levels in list(itertools.combinations(levels, 3))]).unique().tolist()
squad_coop_levels = pd.Series([np.sum(levels) for levels in list(itertools.combinations(levels, 4))]).unique().tolist()

res = np.sort(pd.Series(duo_coop_levels + tripple_coop_levels + squad_coop_levels).unique())

res

array([ 6,  8, 10, 12, 14, 16, 18, 20])

### Tests

In [8]:
episode_over = False
step_amount = 0
while not episode_over:
    tr_marla_class.update_observation(observation[0])
    actions = tr_marla_class.agents_choose_actions()
    observation, reward, terminated, truncated, info = env.step(tuple(actions))
    # let 2 senconds pass
    env.render()
    time.sleep(0.2)
    episode_over = terminated or truncated
    step_amount += 1
env.close()
print(f"Game lasted {step_amount} steps")

Game lasted 18 steps


In [ ]:
tr_marla_class.update_observation(observation[0])
actions = tr_marla_class.agents_choose_actions()
observation, reward, terminated, truncated, info = env.step(tuple(actions))
tr_marla_class.full_info_field